In [329]:
import plotly.express as px
import pandas as pd
import math

In [330]:
m_b = 120810895044353150938886048668570711901
m_s = 100000000000000000000000000000000000000
min_delta_b = 1647
min_delta_s = 1
max_delta_b = 9859
max_delta_s = 2081
slopes = [[
        (1722, 2081, 1.208478513356562),
        (1721, 2080, 1.208599651365485),
        (1720, 2078, 1.208139534883721),
        (1719, 2077, 1.20826061663758),
        (1718, 2076, 1.2083818393480792),
        (1717, 2075, 1.2085032032615026),
        (1716, 2074, 1.2086247086247086),
        (1715, 2072, 1.2081632653061225),
        (1714, 2071, 1.2082847141190198),
        (1713, 2070, 1.2084063047285465),
        (1712, 2069, 1.2085280373831775),
        (1711, 2068, 1.2086499123319696),
        (1710, 2066, 1.208187134502924),
        (1709, 2065, 1.208308952603862),
        (1708, 2064, 1.2084309133489461),
        (1707, 2063, 1.2085530169888694),
        (1706, 2062, 1.208675263774912),
        (1705, 2060, 1.2082111436950147),
        (1704, 2059, 1.2083333333333333),
        (1703, 2058, 1.2084556664709336),
        (1702, 2057, 1.208578143360752),
        (1701, 2055, 1.2081128747795415),
],[
        (1701, 2055, 1.2081128747795415),
        (1686, 2037, 1.208185053380783),
        (1672, 2020, 1.208133971291866),
        (1657, 2002, 1.2082076041038021),
        (1647, 1990, 1.2082574377656345),
]]

In [331]:
perfect = []
best_imperfect = []
other_imperfect = []
delta_b_too_low = []
suboptimal_delta_b = []
prev_delta_b = None
best_imperfect_price = None
best_imperfect_interval = []
for delta_s in range(min_delta_s, max_delta_s + 1):
    delta_b = math.floor(delta_s * m_s / m_b)
    if (delta_b == 0):
        print("delta_b == 0 for", delta_s)
        continue
    price = delta_s / delta_b
    if (delta_b > max_delta_b):
        delta_b = max_delta_b
    if (delta_b < min_delta_b):
        delta_b_too_low.append((delta_b, delta_s, price))
        continue
    if (delta_b == prev_delta_b):
        suboptimal_delta_b.append((delta_b, delta_s, price))
        continue
    prev_delta_b = delta_b
    if delta_b * m_b == delta_s * m_s:
      perfect.append((delta_b, delta_s, price))
      best_imperfect += best_imperfect_interval
      best_imperfect_price = None
      best_imperfect_interval = []
    elif (best_imperfect_price is None):
      best_imperfect_price = price
      best_imperfect_interval.append((delta_b, delta_s, price))
    elif (price == best_imperfect_price):
      best_imperfect_interval.append((delta_b, delta_s, price))
    elif (price < best_imperfect_price):
      best_imperfect_price = price
      other_imperfect += best_imperfect_interval
      best_imperfect_interval = [(delta_b, delta_s, price)]
    else:
      other_imperfect.append((delta_b, delta_s, price))
    if (delta_b == max_delta_b):
        break
    
best_imperfect += best_imperfect_interval

delta_b == 0 for 1


In [332]:
# Convert your data lists to Pandas DataFrames for easier handling
df1 = pd.DataFrame(perfect, columns=['delta_b', 'delta_s', 'price'])
df2 = pd.DataFrame(best_imperfect, columns=['delta_b', 'delta_s', 'price'])
df3 = pd.DataFrame(other_imperfect, columns=['delta_b', 'delta_s', 'price'])
df4 = pd.DataFrame(delta_b_too_low, columns=['delta_b', 'delta_s', 'price'])
df5 = pd.DataFrame(suboptimal_delta_b, columns=['delta_b', 'delta_s', 'price'])
# df6 = pd.DataFrame(first_slope, columns=['delta_b', 'delta_s', 'price'])
# df7 = pd.DataFrame(second_slope, columns=['delta_b', 'delta_s', 'price'])
slopes_df = [pd.DataFrame(slope, columns=['delta_b', 'delta_s', 'price']) for slope in slopes]
for i in range(len(slopes_df)):
    slopes_df[i]['Series'] = 'slope' + str(i + 1)

# Combine the dataframes with an identifier column
df1['Series'] = 'Without'
df2['Series'] = 'best'
df3['Series'] = 'other'
# df4['Series'] = 'Delta_b too low'
df5['Series'] = 'subopt'
# df6['Series'] = '1st'
# df7['Series'] = '2nd'
# pd.concat([df5, df1, df3] + slopes_df + [df2])
combined_df = pd.concat([df5, df1, df3, df2]) 

# Add a new column for point size based on the series
# Assuming 'Series' is a column that identifies each series
# series_size_mapping = {series: i+1 for i, series in enumerate(combined_df['Series'].unique())}
# combined_df['point_size'] = combined_df['Series'].map(series_size_mapping)

# Sort the DataFrame in reverse order of point size
# combined_df = combined_df.sort_values(by='point_size', ascending=False)

# Create an interactive scatter plot
fig = px.scatter(combined_df, x='delta_s', y='price', color='Series', #size='point_size',
                 hover_data=['delta_b', 'delta_s', 'price'])

# Add lines for specified series
# for series in slopes_df:
#     fig.add_trace(px.line(series, x='delta_s', y='price', color='Series').data[0])

# Concatenate all DataFrames in slopes_df
all_slopes_df = pd.concat(slopes_df)
# Create a line plot for all slopes
line_fig = px.line(all_slopes_df, x='delta_s', y='price', color='Series', hover_data=['delta_b', 'delta_s', 'price'])

# Add each line trace individually
for trace in line_fig.data:
    fig.add_trace(trace)

# Update marker properties to have darker outlines
# fig.update_traces(marker=dict(line=dict(width=1, color='rgba(40,40,40,1)')))  # Adjust the color as needed

# Update marker properties to remove outlines or match the fill color
fig.update_traces(marker=dict(line=dict(width=0)))  # Removes the outline

# Alternatively, to set the outline to the same color as the fill:
# fig.update_traces(marker=dict(line=dict(width=1, color='SAME_COLOR')))


# Update the layout for dark mode
fig.update_layout(
    title='Scatter Plot in Dark Mode',
    plot_bgcolor='rgb(17,17,17)',  # Dark plot background
    paper_bgcolor='rgb(17,17,17)',  # Dark paper background
    font_color='white',  # White text for better contrast
    xaxis=dict(
        gridcolor='rgb(80,80,80)',  # Darker gridlines
    ),
    yaxis=dict(
        gridcolor='rgb(80,80,80)',  # Darker gridlines
    )
)

# Add plot titles and labels
fig.update_layout(title='effective price by delta selling',
                  xaxis_title='Delta_s',
                  yaxis_title='Price',
                  # showlegend=False,
                  height=1000)
                  

# fig.update_yaxes(range=[combined_df['price'].min(), combined_df['price'].max()])  # Adjust the range as needed

# # Sort the DataFrame by 'ratio' in descending order
# sorted_df = combined_df.sort_values(by='price', ascending=False)

# # Adding a line that connects the dots in decreasing order of their y-value
# fig.add_trace(px.line(sorted_df, x='delta_b', y='price').data[0])

# # Find the five lowest unique y-values (ratios)
# lowest_ratios = combined_df['price'].unique()
# lowest_ratios.sort()

# # Group by 'ratio' and draw lines for each of the five lowest ratios
# for price in lowest_ratios[:20]:
#     group = combined_df[combined_df['price'] == price]
#     # Sort each group by 'delta_b' to make line connections logical
#     sorted_group = group.sort_values(by='delta_b')
#     fig.add_trace(px.line(sorted_group, x='delta_b', y='price').data[0])

# # Adjusting the plot size and layout
# fig.update_layout(title='Scatter Plot with Lines for Lowest Y-values',
#                   xaxis_title='Delta_b',
#                   yaxis_title='Price',
#                   height=800)

# Show the plot
fig.show()


In [333]:
9273 - 29

9244

In [334]:
4.331472620946305 - 4.3314728682170545

-2.472707496536941e-07

### Patterns:

- the (imperfect) optimal values appear to have equal distances in both deltas to each other 
  - -> if we're seeing the same value again, we can quit
- similarly for the points within a "slope"
- we can't see any way to compute those distances (see below for our attempts)
  - we could maybe try to measure them somehow:
    - start on the rightmost side
    - record the best price seen
    - go leftwards
    - if the price improves a second time, measure that distance
    - continue going leftwards, using that distance
    - as soon as the price gets worse, we have found the minimum of a "slope" (this is a global optimum in this chart, but not the other one)
    - we record this, and do the same thing again, this time also computing the distance between "slopes"
    - we could use this to efficiently search over the slopes, or simply be content with repeating this until we found a global optimum

In [335]:
11179 - 10940

239

In [336]:
11418 - 11179

239

In [337]:
7250 - 7095

155

In [338]:
7405 - 7250

155

In [339]:
8752 - 6564

2188

In [340]:
10940 - 8752

2188

In [341]:
5676 - 4257

1419

In [342]:
7095 - 5676

1419

In [343]:
def prime_factors(n):
    i = 2
    factors = {}
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            factors[i] = factors.get(i, 0) + 1
    if n > 1:
        factors[n] = factors.get(n, 0) + 1
    return factors

In [344]:
prime_factors(239)

{239: 1}

In [345]:
prime_factors(239 + 1)

{2: 4, 3: 1, 5: 1}

In [346]:
prime_factors(155)

{5: 1, 31: 1}

In [347]:
prime_factors(155 + 1)

{2: 2, 3: 1, 13: 1}

In [348]:
prime_factors(2188)

{2: 2, 547: 1}

In [349]:
prime_factors(2188 + 1)

{11: 1, 199: 1}

In [350]:
prime_factors(1419)

{3: 1, 11: 1, 43: 1}

In [351]:
prime_factors(1419 + 1)

{2: 2, 5: 1, 71: 1}

In [352]:
prime_factors(m_b)

{101: 19}

In [353]:
prime_factors(m_s)

{2: 38, 5: 38}